# Part 3: PPD_ED_Cost (test mode)

This part of the code estimate Peak Power Demand, Electricity demand & Electricity cost for electric or diesel pumps

In [1]:
## import the required Python packages in case you want to run it separately

import pandas as pd
import datetime
import numpy as np
import ast
from ast import literal_eval
from pandas import DataFrame
from datetime import datetime
%matplotlib inline

### Read the input files

In [2]:
# Read the input file from the part 2
df1=pd.read_excel('Pilot_Results.xlsx')

# Read electricity and diesel prices to be tested
df2=pd.read_excel('Pilot_Input_Fuel_Prices.xlsx')
df2.round(decimals=3)
df2.head()

#Merge both dfs on country name column
data = pd.merge(df1, df2, on='country')

### Estimate parameters

In [3]:
%%time

#Setting the default value for these parameters
for i in range (1,13):
    data['PD_D_{}'.format(i)]=0      #PD_D_: Peak Demand (kw) using diesel powered pump. 
    data['PD_E_{}'.format(i)]=0      #PD_E_: Peak Demand (kw) using electric powered pump.
    data['ED_D_{}'.format(i)]=0      #ED_D_: Electricity Demand (kwh) using diesel powered pump. 
    data['ED_E_{}'.format(i)]=0      #ED_E_: Electricity Demand (kwh) using electric powered pump.
    data['Dcons_{}'.format(i)]=0     #Dcons_: Monthly amount of diesel needed to meet the pumping demand
    data['Dcost_{}'.format(i)]=0     #Dcost_: Diesel cost (USD) to run diesel powered pumps.
    data['Ecost_{}'.format(i)]=0     #Ecost_: Electricity cost (USD) using electric powered pump.  
    

#This step of ground water level specification was removed since we have a layer specifying the depth in m:

#tdh(m): total dynamic head
#gw_depth: depth to groundwater (m) + wdd:water drawdown(?) + oap: operating application pressure (m) + pld: pressure loss in distribution(m)
#operating application pressure: 0 m (SU), 30 m (SP), 10 m (DR) , check if it is in 1992?
#pressure loss in distributionL: 1 m (SU), 20% of oap (SP, DR) , check if it is in 1992?

#wdd=0  #water drawdown
#oap=0   #operating application pressure (m) 
#pld=0   #pressure loss in distribution

#data.rename(index=str, columns={'gw_depth': 'gw_m'})

def tdh_gw(row):
    tdh=(row['gw_depth'])  # add +wdd+oap+pld in scenarios
    return tdh

data['tdh_gw'] = data.apply(tdh_gw , axis=1)

#pumping plant efficiencty (%)= fuel efficiency (%) * "power unit eff (%)" * transmission eff (%) * pump eff (%) * 100%
#The Power Unit: can be diesel engine or electric engine. in the first we call it (diesel powered pump) and the second (electric powered pump)
 
#Value obtained from FAO1992, pages 26-27:

# Diesel powered pump
#Worst case: 0.9*0.3*0.9*0.4 ~ 10 % (0.1)
#Best case: 1*0.4*1*0.8 = 32% (0.32)

# Electric powered pump
#Worst case: 0.9*0.75*0.9*0.4 ~ 25% (0.25)
#Best case: 1*0.85*1*0.8 ~ 70% (0.7)

Dpump_plant_eff=0.1

for i in range (1,13):
    PD_D = 'PD_D_{}'.format(i)
    PWD = 'PWD_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    ED_D = 'ED_D_{}'.format(i)

    data[PD_D]=(9.81*(data[PWD]/1000)*data['tdh_gw'])/Dpump_plant_eff
    data[ED_D]=(data[SSWD]*data['tdh_gw']*0.00272)/Dpump_plant_eff


Epump_plant_eff=0.25

for i in range (1,13):
    PD_E = 'PD_E_{}'.format(i)
    PWD = 'PWD_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    ED_E = 'ED_E_{}'.format(i)
    
    data[PD_E]=(9.81*(data[PWD]/1000)*data['tdh_gw'])/Epump_plant_eff
    data[ED_E]=(data[SSWD]*data['tdh_gw']*0.00272)/Epump_plant_eff

    
#fcons (l/kWh): fuel consumption for diesel motors
#fprice ($/l): diesel fuel price
#wcost ($/m3): Water cost (sourse: OSS2015_for better irrigation)

#Diesel motor:
#Dcons=0.09 l/kWh (page 48, FAO1992) and Dprice=$ 1.9 /l
#wcost=0.036 $/m3 NWSAS average cost  DZ:0.036, TN:0.04, LY:0.028

#Calculating the amount of diesel consumed and the total cost

Dreq= 0.09   #Dreq: Diesel Requirement is the amount of Diesel needed to generate 1 KWh of electricity. It can be called Consumption but I changed it to avoid confusion 

for i in range (1,13):
    ED_D = 'ED_D_{}'.format(i)
    Dcons = 'Dcons_{}'.format(i)
    data[Dcons]=data[ED_D]*Dreq
    

for i in range (1,13):
    Dcost = 'Dcost_{}'.format(i)
    Dcons = 'Dcons_{}'.format(i)
    data[Dcost] = data[Dcons]*data['Dprice1']
    
#Calculating monthly electricity cost:

Econs= 1
#Eprice= 0.11

for i in range (1,13):
    Ecost = 'Ecost_{}'.format(i)
    ED_E = 'ED_E_{}'.format(i)
    data[Ecost] = data[ED_E]*Econs*data['Eprice1']
    
final=data.groupby('FID', as_index=False).sum() #changed from .max() instead of .sum()

sswd = final.filter(like='SSWD_').sum()
powerdemandD = final.filter(like='PD_D_').sum()
powerdemandE = final.filter(like='PD_E_').sum()
energydemandD = final.filter(like='ED_D_').sum()
energydemandE = final.filter(like='ED_E_').sum()
dcost = final.filter(like='Dcost_').sum()
dcons = final.filter(like='Dcons_').sum()
ecost = final.filter(like='Ecost_').sum()


# Finally, print results of part 3


#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Pilot_Result_Part3.xlsx', engine='xlsxwriter')
writer.book.use_zip64() 

# Convert the dataframe to an XlsxWriter Excel object.
data.to_excel(writer, sheet_name='sur_ref')
final.to_excel(writer, sheet_name='results aggregated')
sswd.to_excel(writer, sheet_name='SSWD in m3')
powerdemandD.to_excel(writer, sheet_name='DP peak power demand in kw')
powerdemandE.to_excel(writer, sheet_name='EP peak power demand in kw')
energydemandD.to_excel(writer, sheet_name='DP electricity demand in kwh')
energydemandE.to_excel(writer, sheet_name='EP electricity demand in kwh')
ecost.to_excel(writer, sheet_name='electricity cost in USD')
dcons.to_excel(writer, sheet_name='diesel consumption in Liter')
dcost.to_excel(writer, sheet_name='diesel cost in USD')


# Close the Pandas Excel writer and output the Excel file.
writer.save()

Wall time: 13.2 s
